<h1 align= "center"><b>Problem: Escape Pods</b></h1>

You've blown up the LAMBCHOP doomsday device and broken the bunnies out of Lambda's prison - and now you need to escape from the space station as quickly and as orderly as possible! The bunnies have all gathered in various locations throughout the station, and need to make their way towards the seemingly endless amount of escape pods positioned in other parts of the station. You need to get the numerous bunnies through the various rooms to the escape pods. Unfortunately, the corridors between the rooms can only fit so many bunnies at a time. What's more, many of the corridors were resized to accommodate the LAMBCHOP, so they vary in how many bunnies can move through them at a time. 

Given the starting room numbers of the groups of bunnies, the room numbers of the escape pods, and how many bunnies can fit through at a time in each direction of every corridor in between, figure out how many bunnies can safely make it to the escape pods at a time at peak.

Write a function solution(entrances, exits, path) that takes an array of integers denoting where the groups of gathered bunnies are, an array of integers denoting where the escape pods are located, and an array of an array of integers of the corridors, returning the total number of bunnies that can get through at each time step as an int. The entrances and exits are disjoint and thus will never overlap. The path element path[A][B] = C describes that the corridor going from A to B can fit C bunnies at each time step.  There are at most 50 rooms connected by the corridors and at most 2000000 bunnies that will fit at a time.

For example, if you have:

    entrances = [0, 1]
    exits = [4, 5]
    path =
    [
        [0, 0, 4, 6, 0, 0],  # Room 0: Bunnies
        [0, 0, 5, 2, 0, 0],  # Room 1: Bunnies
        [0, 0, 0, 0, 4, 4],  # Room 2: Intermediate room
        [0, 0, 0, 0, 6, 6],  # Room 3: Intermediate room
        [0, 0, 0, 0, 0, 0],  # Room 4: Escape pods
        [0, 0, 0, 0, 0, 0],  # Room 5: Escape pods
    ]

Then in each time step, the following might happen:

    0 sends 4/4 bunnies to 2 and 6/6 bunnies to 3
    1 sends 4/5 bunnies to 2 and 2/2 bunnies to 3
    2 sends 4/4 bunnies to 4 and 4/4 bunnies to 5
    3 sends 4/6 bunnies to 4 and 4/6 bunnies to 5

So, in total, 16 bunnies could make it to the escape pods at 4 and 5 at each time step.  (Note that in this example, room 3 could have sent any variation of 8 bunnies to 4 and 5, such as 2/6 and 6/6, but the final solution remains the same.)


<h3 align= "center"><b>Test Cases</b></h3>

```

Input:
    solution.solution([0], [3], [[0, 7, 0, 0], [0, 0, 6, 0], [0, 0, 0, 8], [9, 0, 0, 0]])
Output:
    6

Input:
    solution.solution([0, 1], [4, 5], [[0, 0, 4, 6, 0, 0], [0, 0, 5, 2, 0, 0], [0, 0, 0, 0, 4, 4], [0, 0, 0, 0, 6, 6], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
Output:
    16

```

<h3 align= "center"><b>The idea</b></h3>

I was stuck trying to solve this problem on my own and decided to look for hints. Googling for words like graph, flow, network led me to learn about the <a href="https://en.wikipedia.org/wiki/Maximum_flow_problem">maximum flow problem</a> and the many different solutions. I chose one which runs in O(V^3) because the number of bunnies could be very large but V is limited to 50 (reference in the code below). Note that the classical problem has only one entrance and one exit. I adapted it to this case in which we have many entrances by adding an extra entrance and letting an infinite flow from it to all the entrances, and similarly for the exits.

<h3 align= "center"><b>The code</b></h3>

In [2]:
def solution(entrances, exits, path):
    
    # Maximum Flow - Push-relabel algorithm in O(V^3)
    # Adapted to Python and multiple sources and targets.
    # reF: https://cp-algorithms.com/graph/push-relabel-faster.html
    
    INFTY = 4000000
    
    n = len(path)
    
    for row in path:
        row.extend([0,0])
    path.append ( path[exits[0]][:] )
    path.append ( path[exits[0]][:] )
    src = -2
    tgt = -1
    for e in entrances:
        path[src][e] = INFTY//2
    for e in exits:
        path[e][tgt] = INFTY//2
    

    def push(u, v, path, flow, excess):
        d = min(excess[u], path[u][v] - flow[u][v])
        flow[u][v] += d
        flow[v][u] -= d
        excess[u] -= d
        excess[v] += d
    
    def relabel(u, path, flow, height):
        d = INFTY
        for i in range(n+2):
            if path[u][i] > flow[u][i]:
                d = min(d, height[i])
        if d < INFTY:
            height[u] = d + 1
    
    def find_max_height_vertices(height, excess):
        max_height = []
        for i in range(n):
            if excess[i]:
                if max_height != [] and height[i] > height[max_height[0]]:
                    max_height = []
                if max_height == [] or height[i] == height[max_height[0]]:
                    max_height.append(i)
        return max_height
        
        
    height = [0 for _ in range(n+2)]
    height[src] = n+2
    flow = [ [0 for _ in range(n+2)] for _ in range(n+2) ]
    excess = [0 for _ in range(n+2)]
    excess[src] = INFTY
    
    for i in range(n):
        push(src, i, path, flow, excess)

    current = find_max_height_vertices(height, excess)
    while current != []:
        
        for i in current:
            pushed = False
            for j in range(n+2):
                if not excess[i]:
                    break
                if path[i][j] > flow[i][j] and height[i] == height[j] + 1:
                    push(i, j, path, flow, excess)
                    pushed = True
            
            if not pushed:
                relabel(i, path, flow, height)
                break
        
        current = find_max_height_vertices(height, excess)
    
    return sum( flow[i][tgt] for i in range(n) )

In [3]:
print (solution([0], [3], [[0, 7, 0, 0], [0, 0, 6, 0], [0, 0, 0, 8], [9, 0, 0, 0]]))

6


In [4]:
print (solution([0, 1], [4, 5], [[0, 0, 4, 6, 0, 0], [0, 0, 5, 2, 0, 0], [0, 0, 0, 0, 4, 4], [0, 0, 0, 0, 6, 6], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]))

16
